In [55]:
import spacy
from spacy.matcher import Matcher
from spacy import displacy
from spacy.matcher import DependencyMatcher

nlp = spacy.load('en_core_web_md')

def show_ents(doc):
    if doc.ents:
        for ent in doc.ents:
            print(ent.text+' - '+ent.label_+' - ' +
                  str(spacy.explain(ent.label_)))
    else:
        print('No named entities found.')

In [60]:
doc = nlp("I am not abled to do well")

for token in doc:
    print(token.lemma_)

I
be
not
abled
to
do
well


In [19]:
# Ruler
ruler = nlp.add_pipe("entity_ruler", before="ner")
patterns = [
    {'label': "ORG", "pattern": "Saba Portal"},
    {'label': "ORG", "pattern": "Saba portal"},
    {'label': "ORG", "pattern": "saba portal"},
    {'label': "ORG", "pattern": "Saba"},
    {'label': "ORG", "pattern": "saba"},
]
ruler.add_patterns(patterns)

# Matcher
matcher = Matcher(nlp.vocab)
saba_patterns = [[{"LOWER": "saba"}]]
matcher.add("Saba", saba_patterns)




In [7]:
def scanner(prompt):
    doc = nlp(prompt)
    # doc = nlp("I am not able to login to saba portal.")
    saba_matches = matcher(doc)
    return saba_matches

def ticket(ticket_choice):
    if (ticket_choice == "Yes"):
        print("Creating a ticket now. Please wait for a moment.")
    else:
        print ("Closing the conversation.")


In [30]:
def selfhealer():
    prompt = input("Describe your problem.")
    saba_matches = scanner(prompt)

    if (len(saba_matches) == 0):
        prompt = input(
            "Please specify the domain of the problem being faced. Eg. Saba portal, Learning portal")
        saba_matches = scanner(prompt)

    if (len(saba_matches) == 0):
        ticket_choice = input(
            "We are unable to find an appropriate solution. Do you want to proceed to create a manual ticket? (Yes/No)")
        ticket(ticket_choice)
        
    


In [ ]:
selfhealer()

In [54]:
doc = nlp("I am facing error when accessing saba")
# show_ents(doc)
for sent in doc.sents:
    print (displacy.render(sent, style = "dep"))
    

None


In [148]:
for token in doc:
    print(f"{token.text:{10}} {token.pos_:{10}} ({token.dep_:{10}} {spacy.explain(token.dep_):{25}})  {token.head}")
    

I          PRON       (nsubj      nominal subject          )  am
am         AUX        (ROOT       root                     )  am
unable     ADJ        (acomp      adjectival complement    )  am
to         PART       (aux        auxiliary                )  access
access     VERB       (xcomp      open clausal complement  )  unable
saba       X          (npadvmod   noun phrase as adverbial modifier)  am


In [28]:
doc = nlp("I am not able to login to saba portal.")
show_ents(doc)

saba portal - ORG - Companies, agencies, institutions, etc.


In [53]:
# Dependency Matcher
dep_matcher = DependencyMatcher(nlp.vocab)
doc = nlp("I am facing error when accessing saba.")
dep_pattern = [
    [
        {
            "RIGHT_ID": "anchor_saba",
            "RIGHT_ATTRS": {"LOWER": "saba"}
        },
        {
            "LEFT_ID": "anchor_saba",
            "REL_OP": "<<",
            "RIGHT_ID": "saba_action",
            "RIGHT_ATTRS": {"POS": "VERB"}
        },
        {
            "LEFT_ID": "saba_action",
            "REL_OP": ">",
            "RIGHT_ID": "negation",
            "RIGHT_ATTRS": {"DEP": "neg"}
        },
    ],
    [
        {
            "RIGHT_ID": "anchor_saba",
            "RIGHT_ATTRS": {"LOWER": "saba"}
        },
        {
            "LEFT_ID": "anchor_saba",
            "REL_OP": "<<",
            "RIGHT_ID": "saba_action",
            "RIGHT_ATTRS": {"POS": "VERB"}
        },
        {
            "LEFT_ID": "saba_action",
            "REL_OP": "<<",
            "RIGHT_ID": "aux_pre_neg",
            "RIGHT_ATTRS": {"POS": "AUX"}
        },
        {
            "LEFT_ID": "aux_pre_neg",
            "REL_OP": ">",
            "RIGHT_ID": "aux_neg",
            "RIGHT_ATTRS": {"DEP": "neg"}
        },
    ],
    [
        {
            "RIGHT_ID": "anchor_saba",
            "RIGHT_ATTRS": {"LOWER": "saba"}
        },
        {
            "LEFT_ID": "anchor_saba",
            "REL_OP": "<<",
            "RIGHT_ID": "aux_unable",
            "RIGHT_ATTRS": {"POS": "AUX"}
        },
        {
            "LEFT_ID": "aux_unable",
            "REL_OP": ">",
            "RIGHT_ID": "unable",
            "RIGHT_ATTRS": {"LOWER": "unable"}
        },
        {
            "LEFT_ID": "unable",
            "REL_OP": ">>",
            "RIGHT_ID": "saba_action",
            "RIGHT_ATTRS": {"POS": "VERB"}
        },
    ],
]

dep_matcher.add("saba_dep", patterns=dep_pattern)
dep_matches = dep_matcher(doc)
actions = []
# print(dep_matches)
for match in dep_matches:
    matches = match[1]
    for sub_match in matches:
        # print(doc[sub_match].pos_)
        if ((str(sub_match).lower() not in "saba") and (sub_match not in actions) and (str(doc[sub_match].pos_) == "VERB")):
            actions.append(doc[sub_match])
    # print(str(doc[matches[1]]) + " " + str(doc[matches[0]]))
    # print(str(doc["saba_action"]))
login_doc = nlp("log")
for action in actions:
    print(action)   


In [ ]:
dep_pattern = [
    {
        "RIGHT_ID": "anchor_saba",
        "RIGHT_ATTRS": {"LOWER": "saba"}
    },
    {
        "LEFT_ID": "anchor_saba",
        "REL_OP": "<",
        "RIGHT_ID": "saba_action",
        "RIGHT_ATTRS":
        {
            "POS": {"IN": ["VERB", "ADJ"]},
            "DEP": {"IN": ["pobj", "dobj", "compound"]}
        }
    },
    #    {"LEFT_ID": "Saba", "REL_OP": ">", "RIGHT_ID": "SabaAction2", "RIGHT_ATTRS" : {"POS": "verb", "DEP" : "dobj"}},
]
dep_matcher.add("saba_dep", patterns=[dep_pattern])

dep_matches = dep_matcher(doc)

print(dep_matches)


In [ ]:
dep_matcher = DependencyMatcher(nlp.vocab)
doc = nlp("I am not able to login to saba portal.")

dep_pattern = [
    [
        {
            "RIGHT_ID": "saba_action",
            "RIGHT_ATTRS": {"POS": "VERB"}
        },
        {
            "LEFT_ID": "saba_action",
            "REL_OP": ">>",
            "RIGHT_ID": "saba_org",
            "RIGHT_ATTRS": {"LOWER": "saba"}
        },
        {
            "LEFT_ID": "saba_action",
            "REL_OP": ">",
            "RIGHT_ID": "negation",
            "RIGHT_ATTRS": {"DEP": "neg"}
        },
    ],
    [
        {
            "RIGHT_ID": "saba_action",
            "RIGHT_ATTRS": {"POS": "VERB"}
        },
        {
            "RIGHT_ID": "aux_pre_negation",
            "RIGHT_ATTRS": {"POS": "AUX"}
        },
        {
            "LEFT_ID": "aux_pre_negation",
            "REL_OP": ">>",
            "RIGHT_ID": "saba_action",
            "RIGHT_ATTRS": {"LOWER": "saba"}
        },
        {
            "LEFT_ID": "saba_action",
            "REL_OP": ">>",
            "RIGHT_ID": "saba_org",
            "RIGHT_ATTRS": {"LOWER": "saba"}
        },
        {
            "LEFT_ID": "saba_action",
            "REL_OP": ">>",
            "RIGHT_ID": "aux_pre_negation",
            "RIGHT_ATTRS": {"POS": "AUX"}
        },
        # {
        #     "LEFT_ID": "aux_pre_negation",
        #     "REL_OP": "<",
        #     "RIGHT_ID": "negation_aux",
        #     "RIGHT_ATTRS": { "DEP": "neg" }
        # },
    ]
]
dep_matcher.add("saba_dep", patterns=dep_pattern)
dep_matches = dep_matcher(doc)
action = []
for match in dep_matches:
    matches = match[1]
    if ((str(doc[matches[0]]).lower() not in "saba") and (doc[matches[0]] not in action)):
        action.append(doc[matches[0]])
    print(str(doc[matches[1]]) + " " + str(doc[matches[0]]))

# print(action)
